In [154]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow import keras
from tensorflow.keras import layers
from scikeras.wrappers import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score
import seaborn
from matplotlib import pyplot

# Load the dataset, split features from label, and visualize first 5 entries.
df = pd.read_csv('dataset/train.csv')
features = df.drop(columns = 'Transported')
label = df['Transported']
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [155]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

def preprocess(data):
    # Feature engineering: Cabin encoding and total spent, age group, and path features injection.
    data[['cabin_deck', 'cabin_number', 'cabin_side']] = data['Cabin'].str.split('/', n=3, expand=True)
    data['cabin_number'] = (data['cabin_number']).astype('float')
    data['total_spent'] = data['RoomService']+data['FoodCourt']+data['ShoppingMall']+data['Spa']+data['VRDeck']
    data['age_0-18'] = data['Age'].apply(lambda x: True if x<=18.0 else False)
    data['path'] = data['HomePlanet']+data['Destination']
    # Drop least relevant and engined cabin column
    data.drop(columns=['PassengerId','Name','Cabin'],inplace=True)

    # Split features into numeric and categorical columns
    numeric_data = data.select_dtypes(include=[np.number])
    categorical_data = data.select_dtypes(exclude=[np.number])
    # Categorical data - mode
    columns = categorical_data.columns.to_list()
    categorical_data = pd.DataFrame(SimpleImputer(missing_values=np.nan, strategy='most_frequent').fit_transform(categorical_data),columns = columns)
    categorical_data = pd.DataFrame(OneHotEncoder(sparse_output=False).fit_transform(categorical_data))
    # Numerical data - mean
    columns = numeric_data.columns.to_list()
    numeric_data = pd.DataFrame(SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(numeric_data),columns = columns)
    numeric_data = pd.DataFrame(StandardScaler().fit_transform(numeric_data))
    # Concat
    data = pd.concat([numeric_data,categorical_data], axis = 1)
    return data

features = preprocess(features)

In [156]:
# Split the resampled data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, label, test_size=0.2, random_state=1)

In [157]:
# Define a Keras neural network model
def create_keras_model():
    model = keras.Sequential([
        layers.Dense(128, input_dim=X_train.shape[1], activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

keras_classifier = KerasClassifier(build_fn=create_keras_model, epochs=120, batch_size=64, verbose=1)


In [158]:
# Ensemble Learning (Soft Voting)
models = [
    ("Random Forest", RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)),
    ("Gradient Boosting", GradientBoostingClassifier(n_estimators=100, random_state=1)),
    ("Decision Tree", DecisionTreeClassifier(criterion="entropy", max_depth=4, random_state=1)),
    ("SVM (RBF Kernel)", SVC(kernel="rbf", C=1.0, gamma=0.2, probability=True, random_state=1)),
    ("Logistic Regression", LogisticRegression(random_state=1, max_iter=50000)),
    ("Keras Neural Network", keras_classifier),
]

ensemble = VotingClassifier(models, voting="soft")
# Train the ensemble model
ensemble.fit(X_train, y_train)


Epoch 1/120


d:\Python39\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


109/109 [==============================] - 1s 3ms/step - loss: 0.4892 - accuracy: 0.7591
Epoch 2/120
109/109 [==============================] - 0s 2ms/step - loss: 0.4135 - accuracy: 0.7975
Epoch 3/120
109/109 [==============================] - 0s 3ms/step - loss: 0.4024 - accuracy: 0.8039
Epoch 4/120
109/109 [==============================] - 0s 2ms/step - loss: 0.3962 - accuracy: 0.8070
Epoch 5/120
109/109 [==============================] - 0s 2ms/step - loss: 0.3880 - accuracy: 0.8087
Epoch 6/120
109/109 [==============================] - 0s 2ms/step - loss: 0.3826 - accuracy: 0.8148
Epoch 7/120
109/109 [==============================] - 0s 2ms/step - loss: 0.3779 - accuracy: 0.8194
Epoch 8/120
109/109 [==============================] - 0s 2ms/step - loss: 0.3754 - accuracy: 0.8156
Epoch 9/120
109/109 [==============================] - 0s 2ms/step - loss: 0.3756 - accuracy: 0.8139
Epoch 10/120
109/109 [==============================] - 0s 2ms/step - loss: 0.3661 - accuracy: 0.8205
E

VotingClassifier(estimators=[('Random Forest',
                              RandomForestClassifier(max_depth=5,
                                                     random_state=1)),
                             ('Gradient Boosting',
                              GradientBoostingClassifier(random_state=1)),
                             ('Decision Tree',
                              DecisionTreeClassifier(criterion='entropy',
                                                     max_depth=4,
                                                     random_state=1)),
                             ('SVM (RBF Kernel)',
                              SVC(gamma=0.2, probability=True, random_state=1)),
                             ('Logistic Regression',
                              LogisticRegression(max_iter=50000,
                                                 random_state=1)),
                             ('Keras Neural Network',
                              KerasClassifier(batch_size=64, build_fn=<function create_keras_model at 0x0000023D156713A0>, epochs=120))],
                 voting='soft')

In [159]:
X_test = pd.read_csv('dataset/test.csv')
X_test = preprocess(X_test)

In [160]:
# Evaluate the ensemble model on the validation set
ensemble_accuracy = accuracy_score(y_val, ensemble.predict(X_val))
print(f"Ensemble Model Accuracy: {ensemble_accuracy}")

# Get probability estimates for test data
test_probabilities = ensemble.predict_proba(X_test)[:, 1]
print(test_probabilities)

28/28 [==============================] - 0s 1ms/step
Ensemble Model Accuracy: 0.8182863714778609
67/67 [==============================] - 0s 1ms/step
[0.60352458 0.06350984 0.9566669  ... 0.90374423 0.67667278 0.62981105]


In [161]:
# Apply a threshold to convert probabilities to binary predictions
test_data = pd.read_csv('dataset/test.csv')
threshold = 0.5
test_predictions = (test_probabilities > threshold).astype(bool)

# Save the test predictions to a CSV file
output = pd.DataFrame({"PassengerId": test_data["PassengerId"], "Transported": test_predictions})
output.to_csv("submission_ensemble.csv", index=False)
print("Test predictions saved to submission_ensemble.csv")

Test predictions saved to submission_ensemble.csv
